# PROJETO FINAL: TURMA 01 - GRUPO 10

# Integrantes:

i29089 - Nuno Pimentel

i31332 - Jesus Alban

i31433 - Matheus Anjos

i31476 - Diogo Gomes

# Objetivos:
Análise de tendencias relativamente à venda de jogos em diversas plataformas e em diversos países

Dataset link: https://www.kaggle.com/datasets/ashaheedq/video-games-sales-2019


In [70]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [71]:
#Variaveis Globais

## Ficheiro CSV de importação
csvFile = 'vgsales-12-4-2019.csv'

## Variaveis a adicionar ao ficheiro de exportação
excelFileName = 'Games_Sales_Analysis.xlsx'
tab1name = 'Filtered Data'
tab2name = 'Global Sales - Platform'
tab3name = 'Global Sales - Genre'
tab4name = 'Global Sales - Publisher'
tab5name = 'Sales per Region - Platform'
tab6name = 'Sales per Region - Genre'
tab7name = 'Sales per Region - Publisher'

# Configuração Pandas CSV
encodingName = 'utf8'
separator = ','

# Configuração do Pandas XLS
engineName = 'xlsxwriter'

# lista de colunas para a limpeza e pre-processamento dos dados
list_columns_delete = ['Rank','basename','VGChartz_Score','Critic_Score','User_Score','Total_Shipped','Last_Update','url','status','Vgchartzscore','img_url','ESRB_Rating']
list_columns_delete_lines_ifnull = ['Publisher','Platform','Publisher','Developer','Global_Sales','Year']
list_columns_fill_nulls = ['NA_Sales', 'PAL_Sales', 'JP_Sales', 'Other_Sales']

# Definições gráficas
top_20_sum_platform = 'top_20_sum_platform.png'
top_20_average_platform = 'top_20_average_platform.png'
sum_genre = 'sum_genre.png'
average_genre = 'average_genre.png'
percentage_genre = 'percentage_genre.png'
top_10_sum_publisher = 'top_10_sum_publisher.png'
top_10_average_publisher = 'top_10_average_publisher.png'
sum_region_platform = 'sum_region_platform.png'
average_region_platform = 'average_region_platform.png'
sum_region_genre = 'sum_region_genre.png'
average_region_genre = 'average_region_genre.png'
sum_region_top20_publisher = 'sum_region_top20_publisher.png'
average_region_top20_publisher = 'average_region_top20_publisher.png'


# DEFINIR FUNÇÕES

Definição das funções relevantes para os objetivos propostos para o projeto final.

Considerámos as seguintes funções:

1. Remover indice inicial de um dataframe
2. Eliminar uma ou mais colunas de um dataframe
3. Eliminar/filtrar linhas com valores nulos de uma ou mais colunas
4. Converter os valores de uma coluna em integer
5. Preencher vazios de uma ou mais colunas
6. Calcular a soma dos valores de uma coluna por grupo de colunas
7. Calcular a média dos valores de uma coluna por grupo de colunas
8. Calcular a percentagem dos valores de uma coluna por grupo de colunas
9. Criar um gráfico de linhas
10. Criar um gráfico de barras
11. Criar um piechart


In [72]:
# Função para remover a coluna de index de uma dataframe
def transform_remove_index(df):
    return df.drop(df.columns[0], axis=1)

In [73]:
# Função para eliminar colunas de um dataframe

def delete_columns(df, *columns_to_delete):
    columns_list = list(columns_to_delete)
    df = df.drop(columns=columns_list, errors='ignore')
    return df

In [74]:
# Função que filtra o dataframe com base nos valores não nulos de uma ou mais colunas

def filter_not_null(df,*columns_filter):
    try:
        filtered_df = df
        columns_list = list(columns_filter)
        for column in columns_list:
            if column in df.columns:
                filtered_df = filtered_df[filtered_df[column].notnull()]
        return filtered_df
    except:
        print(f'Column {column} not in DataFrame')

In [75]:
# Função que converte os valores de uma ou mais colunas em integer

def convert_int(df,*inColumn):
    columns_list = list(inColumn)
    for column in columns_list:
        if column in df.columns:
            df[column] = df[column].astype(int)
    return df

In [76]:
# Função que preenche os vazios de uma ou mais colunas

def fill_null(df, *column_name ,fill_value=0):
    for column in column_name:
        if column in df.columns:
            df[column] = df[column].fillna(fill_value)
    return df

In [77]:
# Função que soma os valores de uma coluna e agrupa por uma ou mais colunas
# O reset index cria indexes para a nova lista criada
# A função sort_values ordena os dados por ordem descendente da soma
# No final é renomeada a coluna da soma para o nome que se quiser

def sum_column_by_group(df, inColumn, *group_by_column, sum_column_name='Sum'):
    group_by_list = list(group_by_column)
    df_sum_grouped = df.groupby(group_by_list)[inColumn].sum().reset_index()
    
    df_sum_sorted = df_sum_grouped.sort_values(by=inColumn, ascending = False).reset_index(drop=True)
    df_sum_sorted.rename(columns={inColumn: sum_column_name}, inplace = True)
    return df_sum_sorted

In [78]:
# Função que cacula a média dos valores de uma coluna e agrupa por uma ou mais colunas
# O reset index cria indexes para a nova lista criada
# A função sort_values ordena os dados por ordem descendente da média
# No final é renomeada a coluna da média para o nome que se quiser

def mean_column_by_group(df, inColumn, *group_by_column, avg_column_name='Average'):
    group_by_list = list(group_by_column)
    df_mean_grouped = df.groupby(group_by_list)[inColumn].mean().reset_index()
    
    df_mean_grouped[inColumn] = df_mean_grouped[inColumn].round(2)
                                                    
    df_mean_sorted = df_mean_grouped.sort_values(by=inColumn, ascending = False).reset_index(drop=True)
    df_mean_sorted.rename(columns={inColumn: avg_column_name}, inplace = True)
    return df_mean_sorted

In [79]:
# Função que calcula a percentagem dos valores de uma coluna em função de outra coluna

def percentage_by_group(df,inColumn,group_by_column, per_column_name='percentage'):
    total_sum = df[inColumn].sum()
    Sum_per_group = sum_column_by_group(df, inColumn, group_by_column, sum_column_name=per_column_name)
    i = 0    
    while i < len(Sum_per_group):
        Sum_per_group.loc[i, per_column_name] = round(Sum_per_group.loc[i, per_column_name] / total_sum * 100,2)
        i += 1

    Sum_per_group = Sum_per_group.sort_values(by=per_column_name, ascending = False).reset_index(drop=True)

    return Sum_per_group


In [80]:
# Função com a configuração para o plot de um gráfico de linhas

def plot_trends(df, x_column, y_column, hue_column, title_name, xlabel_name, ylabel_name, marker_type='o', show_legend=False, legend_name=None, image_name='line_plot.png',plotsize=(12, 6)):
    # Define o tamanho da área de plot
    plt.figure(figsize=plotsize)
    
    # Gere cores distintas
    colors = plt.cm.tab20.colors

    # Cria o gráfico de linhas
    sns.lineplot(data=df, x=x_column, y=y_column, hue=hue_column, marker=marker_type, color=colors)
    
    # titulos e labels
    plt.title(title_name)
    plt.xlabel(xlabel_name)
    plt.ylabel(ylabel_name)
    plt.xticks(rotation=25)

    # Condição se há legenda
    if show_legend and legend_name is not None:
        plt.legend(title=legend_name)

    # Grava com um nome e mostra
    plt.savefig(image_name, bbox_inches='tight', dpi=300)
    plt.show()

In [81]:
# Função com a configuração para o plot de um gráfico de barras

def plot_trends_bar(df, x_column, y_column, title_name, xlabel_name, ylabel_name, show_legend=False, legend_name=None, image_name='bar_plot.png',plotsize=(12, 6)):
    # Define o tamanho da área de plot
    plt.figure(figsize=plotsize)

    # Gere cores distintas
    colors = plt.cm.tab20.colors
    
    # Cria o gráfico de barras
    sns.barplot(x=x_column,y=y_column,data=df, color=colors[0])
    
    # Titulos e labels
    plt.title(title_name)
    plt.xlabel(xlabel_name)
    plt.ylabel(ylabel_name)
    plt.xticks(rotation=25)

    # Condição se há legenda
    if show_legend and legend_name is not None:
        plt.legend(title=legend_name)

    # Grava com um nome e mostra
    plt.savefig(image_name, bbox_inches='tight', dpi=300)
    plt.show()


In [82]:
# Função com a configuração para plot de um piechart

def plot_pie_chart(df, labels_column, values_column, title_name, show_legend=False, legend_name=None, image_name='pie_plot.png', plotsize=(10, 6)):
    # Define o tamanho da área de plot
    plt.figure(figsize=plotsize)

    # Gere cores distintas
    colors = plt.cm.tab20.colors

    # Cria o gráfico circular
    wedges = plt.pie(df[values_column], labels=None, autopct='%1.1f%%', startangle=140, colors=colors[:len(df[labels_column])])[0]

    # Titulo
    plt.title(title_name)

    # Condição se há legenda
    if show_legend and legend_name is not None:
        plt.legend(wedges, df[labels_column], title=legend_name, loc='center left', bbox_to_anchor=(1, 0.5))
    
    # Grava com um nome e mostra
    plt.savefig(image_name, bbox_inches='tight', dpi=300)
    plt.show()
    

# Tarefa 1: Extração de dados

Extração dos dados do dataset definido e retirar a primeira coluna dos index.

In [83]:
file = pd.read_csv(csvFile,sep=separator,encoding=encodingName)
file = transform_remove_index(file)

# Tarefa 2:  Limpeza e Pré-Processamento de Dados

Aplicámos as funções criadas para a limpeza e pré-processamento dos dados ao dataframe exportado.

In [84]:
file = delete_columns(file,*list_columns_delete)
file = filter_not_null(file,*list_columns_delete_lines_ifnull)
file = convert_int(file,'Year')
file = fill_null(file, *list_columns_fill_nulls)

# Tarefa 3: Exploratory Data Analysis (EDA)

Com base nos dados existentes no dataset é possivel fazer analisar os dados nas seguintes vertentes:

1. Análise das vendas globais por plataforma: Criar dataframes com a soma e a média total de vendas por plataforma e plotar o top 10 em gráficos de barras
2. Análise das vendas globais por género: criar dataframes com a soma, média e percentagem do total de vendas por género e plotar o top 10 em gráficos de barras e um piechart com as percentagens
3. Análise das vendas globais por publisher: Criar dataframes com a soma e média total de vendas por publisher e plotar o top 10 em gráficos de barras
4. Análise das vendas por região por plataforma: Criar as dataframes para a soma e média para cada região por plataforma, unir as dataframes e plotar o top 10 em gráficos de linhas
5. Análise das vendas por região por género: Criar as dataframes para a soma e média para cada região por género, unir as dataframes e plotar o top 10 em gráficos de linhas
6. Análise das vendas por região por publisher: Criar as dataframes para a soma e média para cada região por publisher, unir as dataframes e plotar o top 10 em gráficos de linhas

VENDAS GLOBAIS POR PLATAFORMA

In [ ]:
# Total de vendas por plataforma
sum_global_by_platform = sum_column_by_group(file, 'Global_Sales', 'Platform', sum_column_name='Total_Sales_M')

# Média de vendas por plataforma
mean_global_by_platform = mean_column_by_group(file, 'Global_Sales', 'Platform', avg_column_name='Average_Sales_M')

# Gráficos para top 10 vendas e top 10 média por plataforma
plot_trends_bar(sum_global_by_platform.head(20), 'Platform', 'Total_Sales_M', 'Top 20 - Global sales per platform', \
                'Platform name', 'Sales (Millions)', image_name=top_20_sum_platform)
plot_trends_bar(mean_global_by_platform.head(20), 'Platform', 'Average_Sales_M', 'Top 20 - Average global sales per platform', \
                'Platform name', 'Average sales (Millions)',image_name=top_20_average_platform)

VENDAS GLOBAIS POR GÉNERO DE JOGO

In [ ]:
# Total de vendas por género
sum_global_by_genre = sum_column_by_group(file, 'Global_Sales', 'Genre', sum_column_name='Total_Sales_M')

# Média de vendas por género
mean_global_by_genre = mean_column_by_group(file, 'Global_Sales', 'Genre', avg_column_name='Average_Sales_M')

# Percentagem de vendas por Género
percentage_by_genre = percentage_by_group(file,'Global_Sales','Genre',per_column_name ='Percentage')

# Gráficos para top 10 vendas e top 10 média por género
plot_trends_bar(sum_global_by_genre, 'Genre', 'Total_Sales_M', 'Global sales per game genre', \
                'Game genre', 'Sales (Millions)',image_name=sum_genre)
plot_trends_bar(mean_global_by_genre, 'Genre', 'Average_Sales_M', 'Average global sales per game genre', \
                'Game genre', 'Average sales (Millions)',image_name=average_genre)
plot_pie_chart(percentage_by_genre, 'Genre', 'Percentage', 'Percentage of total sales per game genre',show_legend=True,legend_name='Genre',image_name=percentage_genre)


VENDAS GLOBAIS POR PUBLISHER

In [ ]:
# 3. VENDAS GLOBAIS POR PUBLISHER

# Total de vendas por publisher
sum_global_by_publisher = sum_column_by_group(file, 'Global_Sales', 'Publisher', sum_column_name='Total_Sales_M')
# Média de vendas entre as 10 plataformas que mais venderam jogos
mean_global_by_publisher = mean_column_by_group(file, 'Global_Sales', 'Publisher', avg_column_name='Average_Sales_M')

# Gráficos para top 10 vendas e top 10 média por publisher
plot_trends_bar(sum_global_by_publisher.head(10), 'Publisher', 'Total_Sales_M', 'Top 10 - Global sales per publisher', \
                'Publisher name', 'Sales (Millions)', image_name=top_10_sum_publisher)
plot_trends_bar(mean_global_by_publisher.head(10), 'Publisher', 'Average_Sales_M', 'Top 10 - Average global sales per publisher', \
                'Publisher name', 'Average sales (Millions)',image_name=top_10_average_publisher)


VENDAS POR REGIÃO POR PLATAFORMA

In [ ]:

# Vendas por região por plataforma
sum_NA_by_platform = sum_column_by_group(file, 'NA_Sales', 'Platform', sum_column_name='Total_NA_Sales_M')
sum_PAL_by_platform = sum_column_by_group(file, 'PAL_Sales', 'Platform', sum_column_name='Total_PAL_Sales_M')
sum_JP_by_platform = sum_column_by_group(file, 'JP_Sales', 'Platform', sum_column_name='Total_JP_Sales_M')
sum_Other_by_platform = sum_column_by_group(file, 'Other_Sales', 'Platform', sum_column_name='Total_Other_Sales_M')

# Média de vendas por plataforma
mean_NA_by_platform = mean_column_by_group(file, 'NA_Sales', 'Platform', avg_column_name='Average_NA_Sales_M')
mean_PAL_by_platform = mean_column_by_group(file, 'PAL_Sales', 'Platform', avg_column_name='Average_PAL_Sales_M')
mean_JP_by_platform = mean_column_by_group(file, 'JP_Sales', 'Platform', avg_column_name='Average_JP_Sales_M')
mean_Other_by_platform = mean_column_by_group(file, 'Other_Sales', 'Platform', avg_column_name='Average_Other_Sales_M')

# Merge dos dataframes de média e total de vendas por plataforma por cada região
sum_region_per_platform = sum_NA_by_platform.merge(sum_PAL_by_platform, on = 'Platform').merge(sum_JP_by_platform, on = 'Platform').merge(sum_Other_by_platform, on = 'Platform')
mean_region_per_platform = mean_NA_by_platform.merge(mean_PAL_by_platform, on = 'Platform').merge(mean_JP_by_platform, on = 'Platform').merge(mean_Other_by_platform, on = 'Platform')

# Melte dos dataframes de média e total de vendas por plataforma por região
melted_sum_region_per_platform = sum_region_per_platform.melt(id_vars='Platform', var_name='Region', value_name='Region_Sales')
melted_mean_region_per_platform = mean_region_per_platform.melt(id_vars='Platform', var_name='Region', value_name='Region_average_Sales')

# Gráficos com vendas médias e totais por plataforma por região
plot_trends(melted_sum_region_per_platform, 'Platform', 'Region_Sales', 'Region', 'Sales per Region per platform', \
            'Platform name', 'Sales (Millions)', marker_type='o', show_legend=True, legend_name='Region', image_name=sum_region_platform, plotsize=(24, 6))
plot_trends(melted_mean_region_per_platform, 'Platform', 'Region_average_Sales', 'Region', 'Average sales per Region per platform', \
            'Platform name', 'Average sales (Millions)', marker_type='o', show_legend=True, legend_name='Region', image_name=average_region_platform, plotsize=(24, 6))

VENDAS POR REGIÃO POR GÉNERO DE JOGO

In [ ]:
# Vendas por região por género
sum_NA_by_genre = sum_column_by_group(file, 'NA_Sales', 'Genre', sum_column_name='Total_NA_Sales_M')
sum_PAL_by_genre = sum_column_by_group(file, 'PAL_Sales', 'Genre', sum_column_name='Total_PAL_Sales_M')
sum_JP_by_genre = sum_column_by_group(file, 'JP_Sales', 'Genre', sum_column_name='Total_JP_Sales_M')
sum_Other_by_genre = sum_column_by_group(file, 'Other_Sales', 'Genre', sum_column_name='Total_Other_Sales_M')

# Média de vendas por género
mean_NA_by_genre = mean_column_by_group(file, 'NA_Sales', 'Genre', avg_column_name='Average_NA_Sales_M')
mean_PAL_by_genre = mean_column_by_group(file, 'PAL_Sales', 'Genre', avg_column_name='Average_PAL_Sales_M')
mean_JP_by_genre = mean_column_by_group(file, 'JP_Sales', 'Genre', avg_column_name='Average_JP_Sales_M')
mean_Other_by_genre = mean_column_by_group(file, 'Other_Sales', 'Genre', avg_column_name='Average_Other_Sales_M')

# Merge dos dataframes de média e total de vendas por género por cada região
sum_region_per_genre = sum_NA_by_genre.merge(sum_PAL_by_genre, on = 'Genre').merge(sum_JP_by_genre, on = 'Genre').merge(sum_Other_by_genre, on = 'Genre')
mean_region_per_genre = mean_NA_by_genre.merge(mean_PAL_by_genre, on = 'Genre').merge(mean_JP_by_genre, on = 'Genre').merge(mean_Other_by_genre, on = 'Genre')

# Melte dos dataframes de média e total de vendas por género por região
melted_sum_region_per_genre = sum_region_per_genre.melt(id_vars='Genre', var_name='Region', value_name='Region_Sales')
melted_mean_region_per_genre = mean_region_per_genre.melt(id_vars='Genre', var_name='Region', value_name='Region_average_Sales')

# Gráficos com vendas médias e totais por género por região
plot_trends(melted_sum_region_per_genre, 'Genre', 'Region_Sales', 'Region', 'Sales per Region per game genre', \
            'Game genre', 'Sales (Millions)', marker_type='o', show_legend=True, legend_name='Region', image_name=sum_region_genre, plotsize=(24, 6))
plot_trends(melted_mean_region_per_genre, 'Genre', 'Region_average_Sales', 'Region', 'Average sales per Region per game genre', \
            'Game genre', 'Average sales (Millions)', marker_type='o', show_legend=True, legend_name='Region', image_name=average_region_genre, plotsize=(24, 6))

VENDAS POR REGIÃO POR PUBLISHER

In [ ]:
# Vendas por região por género
sum_NA_by_Publisher = sum_column_by_group(file, 'NA_Sales', 'Publisher', sum_column_name='Total_NA_Sales_M')
sum_PAL_by_Publisher = sum_column_by_group(file, 'PAL_Sales', 'Publisher', sum_column_name='Total_PAL_Sales_M')
sum_JP_by_Publisher = sum_column_by_group(file, 'JP_Sales', 'Publisher', sum_column_name='Total_JP_Sales_M')
sum_Other_by_Publisher = sum_column_by_group(file, 'Other_Sales', 'Publisher', sum_column_name='Total_Other_Sales_M')

# Média de vendas por género
mean_NA_by_Publisher = mean_column_by_group(file, 'NA_Sales', 'Publisher', avg_column_name='Average_NA_Sales_M')
mean_PAL_by_Publisher = mean_column_by_group(file, 'PAL_Sales', 'Publisher', avg_column_name='Average_PAL_Sales_M')
mean_JP_by_Publisher = mean_column_by_group(file, 'JP_Sales', 'Publisher', avg_column_name='Average_JP_Sales_M')
mean_Other_by_Publisher = mean_column_by_group(file, 'Other_Sales', 'Publisher', avg_column_name='Average_Other_Sales_M')

# Merge dos dataframes de média e total de vendas por género por cada região
sum_region_per_Publisher = sum_NA_by_Publisher.merge(sum_PAL_by_Publisher, on = 'Publisher').merge(sum_JP_by_Publisher, on = 'Publisher').merge(sum_Other_by_Publisher, on = 'Publisher')
mean_region_per_Publisher = mean_NA_by_Publisher.merge(mean_PAL_by_Publisher, on = 'Publisher').merge(mean_JP_by_Publisher, on = 'Publisher').merge(mean_Other_by_Publisher, on = 'Publisher')

# Melte dos dataframes de média e total de vendas por género por região
melted_sum_region_per_Publisher = sum_region_per_Publisher.melt(id_vars='Publisher', var_name='Region', value_name='Region_Sales')
melted_mean_region_per_Publisher = mean_region_per_Publisher.melt(id_vars='Publisher', var_name='Region', value_name='Region_average_Sales')

# Visualizar apenas os 20 Publishers que mais vendem
list_of_top20_publishers = sum_global_by_publisher['Publisher'].head(20).tolist()

# Filtrar os melted dataframes
top20_melted_sum_region_per_Publisher = melted_sum_region_per_Publisher[melted_sum_region_per_Publisher['Publisher'].isin(list_of_top20_publishers)]
top20_melted_mean_region_per_Publisher = melted_mean_region_per_Publisher[melted_mean_region_per_Publisher['Publisher'].isin(list_of_top20_publishers)]

# Gráficos com vendas médias e totais por região e por top 20 de publisher mais vendidos
plot_trends(top20_melted_sum_region_per_Publisher, 'Publisher', 'Region_Sales', 'Region', 'Sales per Region per Top 20 publisher', \
            'Publisher name', 'Sales (Millions)', marker_type='o', show_legend=True, legend_name='Region', image_name=sum_region_top20_publisher, plotsize=(24, 6))
plot_trends(top20_melted_mean_region_per_Publisher, 'Publisher', 'Region_average_Sales', 'Region', 'Average sales per Region per Top 20 publisher', \
            'Publisher name', 'Average sales (Millions)', marker_type='o', show_legend=True, legend_name='Region', image_name=average_region_top20_publisher, plotsize=(24, 6))

# Tarefa 4: Gravar dados em ficheiro Excel

Grava para o ficheiro Excel os seguintes dados:
1. Dados filtrados definidos na tarefa 2 na sheet "Filtered Data"
2. Dataframes e gráficos criados que analise as global sales por plataforma na Sheet "Global Sales - Platform"
3. Dataframes e gráficos criados que analise as global sales por género de jogo na Sheet "Global Sales - Genre"
4. Dataframes e gráficos criados que analise as global sales por publisher na Sheet "Global Sales - Publisher"
5. Dataframes e gráficos criados que analise das Region Sales por plataforma na Sheet "Region Sales - Platform"
6. Dataframes e gráficos criados que analise as Region sales por género de jogo na Sheet "Region Sales - Genre"
7. Dataframes e gráficos criados que analise as Region sales por publisher na Sheet "Region Sales - Publisher"

In [97]:
with pd.ExcelWriter(excelFileName, engine=engineName) as writer:
    
    # Inserir na sheet Filtered Data
    file.to_excel(writer, sheet_name=tab1name, startrow=0, startcol=0, index=False)  # Grava na coluna A
    
    # Inserir dataframes e gráficos na sheet de Global Sales - Platform
    sum_global_by_platform.to_excel(writer, sheet_name=tab2name, startrow=0, startcol=0, index=False)  # Grava na coluna A
    mean_global_by_platform.to_excel(writer, sheet_name=tab2name, startrow=0, startcol=3, index=False)  # Grava na coluna D
    
    worksheet = writer.sheets[tab2name]
    x_scale = 0.7  # Reduz para 80% do tamanho original
    y_scale = 0.7  # Reduz para 80% do tamanho original
    worksheet.insert_image('H1', top_20_sum_platform, {'x_scale': x_scale, 'y_scale': y_scale})

    worksheet = writer.sheets[tab2name]
    x_scale = 0.7
    y_scale = 0.7
    worksheet.insert_image('H21', top_20_average_platform, {'x_scale': x_scale, 'y_scale': y_scale})

    # Inserir dataframes e gráficos na sheet de Global Sales - Genre
    sum_global_by_genre.to_excel(writer,sheet_name=tab3name,startrow=0,startcol=0,index=False) # Grava na coluna A
    mean_global_by_genre.to_excel(writer,sheet_name=tab3name,startrow=0,startcol=3,index=False) # Grava na coluna D
    percentage_by_genre.to_excel(writer,sheet_name=tab3name,startrow=0,startcol=6,index=False) # Grava na coluna G

    worksheet = writer.sheets[tab3name]
    x_scale = 0.7
    y_scale = 0.7
    worksheet.insert_image('J1', sum_genre, {'x_scale': x_scale, 'y_scale': y_scale})

    worksheet = writer.sheets[tab3name]
    x_scale = 0.7
    y_scale = 0.7
    worksheet.insert_image('J22', average_genre, {'x_scale': x_scale, 'y_scale': y_scale})

    worksheet = writer.sheets[tab3name]
    x_scale = 0.8
    y_scale = 0.8
    worksheet.insert_image('U1', percentage_genre, {'x_scale': x_scale, 'y_scale': y_scale})

    # Inserir dataframes e gráficos na sheet de Global Sales - Publisher
    sum_global_by_publisher.to_excel(writer,sheet_name=tab4name,startrow=0,startcol=0,index=False)
    mean_global_by_publisher.to_excel(writer,sheet_name=tab4name,startrow=0,startcol=3,index=False)

    worksheet = writer.sheets[tab4name]
    x_scale = 0.7
    y_scale = 0.7
    worksheet.insert_image('H1', top_10_sum_publisher, {'x_scale': x_scale, 'y_scale': y_scale})

    worksheet = writer.sheets[tab4name]
    x_scale = 0.7
    y_scale = 0.7
    worksheet.insert_image('H24', top_10_average_publisher, {'x_scale': x_scale, 'y_scale': y_scale})

    # Inserir dataframes e gráficos na sheet de Region Sales - Platform
    sum_region_per_platform.to_excel(writer,sheet_name=tab5name,startrow=0,startcol=0,index=False)
    mean_region_per_platform.to_excel(writer,sheet_name=tab5name,startrow=0,startcol=6,index=False)

    worksheet = writer.sheets[tab5name]
    x_scale = 0.7
    y_scale = 0.7
    worksheet.insert_image('M1', sum_region_platform, {'x_scale': x_scale, 'y_scale': y_scale})

    worksheet = writer.sheets[tab5name]
    x_scale = 0.7
    y_scale = 0.7
    worksheet.insert_image('M21', average_region_platform, {'x_scale': x_scale, 'y_scale': y_scale})

    # Inserir dataframes e gráficos na sheet de Region Sales - Genre
    sum_region_per_genre.to_excel(writer,sheet_name=tab6name,startrow=0,startcol=0,index=False)
    mean_region_per_genre.to_excel(writer,sheet_name=tab6name,startrow=0,startcol=6,index=False)

    worksheet = writer.sheets[tab6name]
    x_scale = 0.7
    y_scale = 0.7
    worksheet.insert_image('M1', sum_region_genre, {'x_scale': x_scale, 'y_scale': y_scale})

    worksheet = writer.sheets[tab6name]
    x_scale = 0.7
    y_scale = 0.7
    worksheet.insert_image('M22', average_region_genre, {'x_scale': x_scale, 'y_scale': y_scale})

    # Inserir dataframes e gráficos na sheet de Region Sales - Publisher
    sum_region_per_Publisher.to_excel(writer,sheet_name=tab7name,startrow=0,startcol=0,index=False)
    mean_region_per_Publisher.to_excel(writer,sheet_name=tab7name,startrow=0,startcol=6,index=False)

    worksheet = writer.sheets[tab7name]
    x_scale = 0.7
    y_scale = 0.7
    worksheet.insert_image('M1', sum_region_top20_publisher, {'x_scale': x_scale, 'y_scale': y_scale})

    worksheet = writer.sheets[tab7name]
    x_scale = 0.7
    y_scale = 0.7
    worksheet.insert_image('M24', average_region_top20_publisher, {'x_scale': x_scale, 'y_scale': y_scale})

writer.close()
